Automate Machine Learning with TPOT
===================================

This example shows how [TPOT](https://epistasislab.github.io/tpot/) can be used with Dask.

TPOT is an [automated machine learning](https://en.wikipedia.org/wiki/Automated_machine_learning) library.
It evaluates many scikit-learn pipelines and hyperparameter combinations to find a model that works well for your data. Evaluating all these computations is computationally expensive, but ammenable to parallelism. TPOT can use Dask to distribute these computations on a cluster of machines.

This notebook can be run interactively on the [dask examples binder](https://github.com/dask/dask-examples).
The following video shows a larger version of this notebook on a cluster.

In [1]:
from IPython.display import HTML

HTML('<div style="position:relative;height:0;padding-bottom:56.25%"><iframe src="https://www.youtube.com/embed/uyx9nBuOYQQ?ecver=2" width="640" height="360" frameborder="0" allow="autoplay; encrypted-media" style="position:absolute;width:100%;height:100%;left:0" allowfullscreen></iframe></div>')

In [2]:
!pip install tpot

     |████████████████████████████████| 87 kB 7.9 MB/s             


     |████████████████████████████████| 160 kB 31.0 MB/s            


  Preparing metadata (setup.py) ... - done

 \

 done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11952 sha256=2caf8e163d84efa030d87b69b668de73c9f29571bc88e7b1a7a855346fe67fd7
  Stored in directory: /home/runner/.cache/pip/wheels/a8/bb/8f/6b9328d23c2dcedbfeb8498b9f650d55d463089e3b8fc0bfb2
Successfully built stopit


In [3]:
import tpot
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


## Setup Dask

We first start a Dask client in order to get access to the Dask dashboard, which will provide progress and performance metrics. 

You can view the dashboard by clicking on the dashboard link after you run the cell.

In [4]:
from dask.distributed import Client
client = Client(n_workers=4, threads_per_worker=1)
client

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33083 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:42461 Dashboard: http://127.0.0.1:33083/status,Cluster Workers: 4 Cores: 4 Memory: 7.29 GB


## Create Data

We'll use the digits dataset.
To ensure the example runs quickly, we'll make the training dataset relatively small.

In [5]:
digits = load_digits()

X_train, X_test, y_train, y_test = train_test_split(
    digits.data,
    digits.target,
    train_size=0.05,
    test_size=0.95,
)

These are just small, in-memory NumPy arrays. This example is not applicable to larger-than-memory Dask arrays.

## Using Dask

TPOT follows the scikit-learn API; we specify a `TPOTClassifier` with a few hyperparameters, and then fit it on some data.
By default, TPOT trains on your single machine.
To ensure your cluster is used, specify the `use_dask` keyword.

In [6]:
# scale up: Increase the TPOT parameters like population_size, generations
tp = TPOTClassifier(
    generations=2,
    population_size=10,
    cv=2,
    n_jobs=-1,
    random_state=0,
    verbosity=0,
    config_dict=tpot.config.classifier_config_dict_light,
    use_dask=True,
)

In [7]:
tp.fit(X_train, y_train)

TPOTClassifier(config_dict={'sklearn.cluster.FeatureAgglomeration': {'affinity': ['euclidean',
                                                                                  'l1',
                                                                                  'l2',
                                                                                  'manhattan',
                                                                                  'cosine'],
                                                                     'linkage': ['ward',
                                                                                 'complete',
                                                                                 'average']},
                            'sklearn.decomposition.PCA': {'iterated_power': range(1, 11),
                                                          'svd_solver': ['randomized']},
                            'sklearn.feature_selection.SelectFwe': {'alpha': array([0.

## Learn More

See the [Dask-ML](http://ml.dask.org/) and [TPOT](https://epistasislab.github.io/tpot/) documenation for more information on using Dask and TPOT.